In [1]:
import nltk; nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cbeams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [3]:
# Import Dataset
df = pd.read_json('../raw_data/all_data_df_stripped.json')
print(df.artist.unique())
df.head()

['Adam Devine' 'Adam Hess' 'Adam Hills' 'Adam Sandler' 'Adel Karam'
 'Aisling Bea' 'Al Madrigal' 'Al Murray' 'Al Porter' 'Alan Carr'
 'Alex Edelman' 'Ali Wong' 'Alistair McGowan' 'Amanda Seales'
 'Amy Schumer' 'Andi Osho' 'Andrew Lawrence' 'Andrew Maxwell'
 'Andy Parsons' 'Andy Woodhull' 'Angela Barnes' 'Anjelah Johnson'
 'Anthony Jeselnik' 'Ari Shaffir' 'Arj Barker' 'Arsenio Hall'
 'Aziz Ansari' 'Bert Kreischer' 'Big Jay Oakerson' 'Bill Burr'
 'Bill Hicks' 'Bill Maher' 'Billy Connolly' 'Bo Burnham' 'Bob Monkhouse'
 'Brad Williams' 'Brent Morin' 'Brian Regan' 'Bridget Everett'
 'Cedric The Entertainer' 'Celia Pacquola' 'Chelsea Peretti'
 'Chris Addison' "Chris D'Elia" 'Chris Gethard' 'Chris McCausland'
 'Chris Moyles' 'Chris Ramsey' 'Chris Rock' 'Chris Tucker'
 'Christina Pazsitzky' 'Colin Quinn' 'Craig Ferguson' 'Cristela Alonzo'
 'D.l. Hughley' 'Dan Soder' 'Dana Carvey' 'Dane Baptiste' 'Daniel Tosh'
 'Danny Bhoy' 'Dara Ó Briain' 'Darren Harriott' 'Dave Allen' 'Dave Attell'
 'Dave Cha

,full_transcript,artist,show_name,year,source,artist_birthday,artist_gender,age_then,full_transcript_clean
0,[rock music playing]\n\n[indistinct chatter]\n...,Adam Devine,Best Time Of Our Lives,2019.0,Scraps from the Loft,1983.0,2,36.0,man guy much seat guy jacked fully juiced man ...
1,"strong language. CHEERING Yes, yes, yes! How...",Adam Hess,Live from the BBC,2016.0,BBC,1990.0,2,26.0,yes yes yes lovely apologise state voice went ...
2,"Ladies and gentlemen, please welcome your hos...",Adam Hills,Live at the Apollo Series 9 Episode 4,2013.0,BBC,1970.0,2,43.0,tonight london london live amazing act present...
3,some strong language and adult humour Ladies...,Adam Hills,Live at the Apollo Series 12 Episode 4,2016.0,BBC,1970.0,2,46.0,tonight hair lost bet british paralympic team ...
4,"? CHEERING Hello, Apollo. I am going to start ...",Adam Hills,Live at the Apollo Series 5 Episode 5,2009.0,BBC,1970.0,2,39.0,start saying something probably never heard co...


In [4]:
# Convert to list
data = df.full_transcript_clean.values.tolist()

pprint(data[:1])

['man guy much seat guy jacked fully juiced man excited guy blast tonight '
 'excited excited though adult man excited lock littlekidlevel geeked remember '
 'kid open present vocabulary much mama however guy talked weirdo said thing '
 'mama much mama vocabulary went like face adult lock adultass thing kid '
 'school drop enjoy lunch today made ham cheese question quality meat made '
 'littlekidlevel excited face month child swear mom gave super bouncy ball '
 'birthday guy poor parent cool gave bouncy ball month straight driveway like '
 'snatched midair phrase use youth midair nothing midair adult child caught '
 'midair mom soaring abyss snatched midair midair mom midair midair midair mom '
 'jesus christ course caught midair honey anything catch air would technically '
 'midair caught ground called picking idiot dumb little boy kids– cool kid '
 'still getting level geeked still getting fully geeked elevator long ago big '
 'deal stair stupid stupid elevator crappy day adult weari

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['man', 'guy', 'much', 'seat', 'guy', 'jacked', 'fully', 'juiced', 'man', 'excited', 'guy', 'blast', 'tonight', 'excited', 'excited', 'though', 'adult', 'man', 'excited', 'lock', 'littlekidlevel', 'geeked', 'remember', 'kid', 'open', 'present', 'vocabulary', 'much', 'mama', 'however', 'guy', 'talked', 'weirdo', 'said', 'thing', 'mama', 'much', 'mama', 'vocabulary', 'went', 'like', 'face', 'adult', 'lock', 'adultass', 'thing', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'made', 'ham', 'cheese', 'question', 'quality', 'meat', 'made', 'littlekidlevel', 'excited', 'face', 'month', 'child', 'swear', 'mom', 'gave', 'super', 'bouncy', 'ball', 'birthday', 'guy', 'poor', 'parent', 'cool', 'gave', 'bouncy', 'ball', 'month', 'straight', 'driveway', 'like', 'snatched', 'midair', 'phrase', 'use', 'youth', 'midair', 'nothing', 'midair', 'adult', 'child', 'caught', 'midair', 'mom', 'soaring', 'abyss', 'snatched', 'midair', 'midair', 'mom', 'midair', 'midair', 'midair', 'mom', 'jesus', 'chris

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['man', 'guy', 'much', 'seat', 'guy', 'jacked', 'fully', 'juiced', 'man', 'excited', 'guy', 'blast', 'tonight', 'excited', 'excited', 'though', 'adult', 'man', 'excited', 'lock', 'littlekidlevel', 'geeked', 'remember', 'kid', 'open', 'present', 'vocabulary', 'much', 'mama', 'however', 'guy', 'talked', 'weirdo', 'said', 'thing', 'mama', 'much', 'mama', 'vocabulary', 'went', 'like', 'face', 'adult', 'lock', 'adultass', 'thing', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'made', 'ham', 'cheese', 'question', 'quality', 'meat', 'made', 'littlekidlevel', 'excited', 'face', 'month', 'child', 'swear', 'mom', 'gave', 'super', 'bouncy', 'ball', 'birthday', 'guy', 'poor', 'parent', 'cool', 'gave', 'bouncy', 'ball', 'month', 'straight', 'driveway', 'like', 'snatched', 'midair', 'phrase', 'use', 'youth', 'midair', 'nothing', 'midair', 'adult', 'child', 'caught', 'midair', 'mom', 'soaring', 'abyss', 'snatched', 'midair', 'midair', 'mom', 'midair', 'midair', 'midair', 'mom', 'jesus_christ', 

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['guy', 'much', 'seat', 'guy', 'jack', 'fully', 'juiced', 'man', 'excited', 'guy', 'blast', 'tonight', 'excited', 'excited', 'adult', 'man', 'excite', 'lock', 'littlekidlevel', 'geeke', 'remember', 'kid', 'open', 'present', 'vocabulary', 'much', 'however', 'talk', 'say', 'thing', 'much', 'go', 'face', 'adult', 'lock', 'adultass', 'thing', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'make', 'ham', 'cheese', 'question', 'quality', 'meat', 'make', 'littlekidlevel', 'excited', 'face', 'month', 'child', 'swear', 'mom', 'give', 'super', 'bouncy', 'ball', 'birthday', 'guy', 'poor', 'parent', 'give', 'bouncy', 'ball', 'month', 'straight', 'driveway', 'snatch', 'midair', 'phrase', 'use', 'youth', 'midair', 'adult', 'child', 'catch', 'mom', 'soar', 'abyss', 'snatch', 'midair', 'mom', 'mom', 'course', 'catch', 'midair', 'honey', 'catch', 'air', 'technically', 'midair', 'catch', 'ground', 'call', 'pick', 'idiot', 'dumb', 'little', 'boy', 'kid', 'cool', 'kid', 'still', 'get', 'level', 'gee

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 9), (1, 2), (2, 1), (3, 1), (4, 1), (5, 4), (6, 1), (7, 1), (8, 6), (9, 1), (10, 3), (11, 1), (12, 10), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 5), (19, 6), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 15), (26, 2), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 3), (34, 8), (35, 1), (36, 1), (37, 4), (38, 1), (39, 3), (40, 2), (41, 1), (42, 8), (43, 1), (44, 9), (45, 3), (46, 1), (47, 8), (48, 1), (49, 1), (50, 5), (51, 3), (52, 5), (53, 1), (54, 4), (55, 1), (56, 3), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 8), (67, 2), (68, 4), (69, 1), (70, 2), (71, 1), (72, 7), (73, 3), (74, 13), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 2), (83, 7), (84, 1), (85, 1), (86, 1), (87, 1), (88, 5), (89, 7), (90, 2), (91, 1), (92, 3), (93, 1), (94, 1), (95, 10), (96, 1), (97, 2), (98, 1), (99, 2), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (105, 1), (106, 3), (107, 7), (108, 1), (109, 1), (110

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(25,
  '0.000*"man" + 0.000*"say" + 0.000*"guy" + 0.000*"good" + 0.000*"shit" + '
  '0.000*"come" + 0.000*"people" + 0.000*"time" + 0.000*"love" + 0.000*"mean"'),
 (4,
  '0.000*"people" + 0.000*"guy" + 0.000*"time" + 0.000*"say" + 0.000*"shit" + '
  '0.000*"good" + 0.000*"day" + 0.000*"come" + 0.000*"talk" + 0.000*"man"'),
 (3,
  '0.000*"say" + 0.000*"come" + 0.000*"time" + 0.000*"people" + 0.000*"guy" + '
  '0.000*"call" + 0.000*"back" + 0.000*"baby" + 0.000*"year" + 0.000*"shit"'),
 (27,
  '0.000*"man" + 0.000*"people" + 0.000*"guy" + 0.000*"say" + 0.000*"shit" + '
  '0.000*"kid" + 0.000*"mean" + 0.000*"talk" + 0.000*"come" + 0.000*"time"'),
 (20,
  '0.000*"man" + 0.000*"say" + 0.000*"people" + 0.000*"time" + 0.000*"love" + '
  '0.000*"little" + 0.000*"never" + 0.000*"shit" + 0.000*"good" + 0.000*"guy"'),
 (29,
  '0.000*"say" + 0.000*"people" + 0.000*"man" + 0.000*"time" + 0.000*"guy" + '
  '0.000*"woman" + 0.000*"shit" + 0.000*"little" + 0.000*"way" + 0.000*"kid"'),
 (24,
  '0.000*

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.2740267159398

Coherence Score:  0.26134747184244106


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.204202  0.377616       1        1  66.563588
26     0.334978  0.341108       2        1  14.397275
13     0.005448  0.481236       3        1   9.641483
9      0.113521  0.511503       4        1   6.896445
7      0.438205  0.095081       5        1   2.412152
17     0.029980 -0.131406       6        1   0.063915
12    -0.069248 -0.047889       7        1   0.015145
28    -0.045960 -0.070750       8        1   0.000474
23    -0.045960 -0.070750       9        1   0.000466
2     -0.045960 -0.070750      10        1   0.000466
8     -0.045960 -0.070750      11        1   0.000463
18    -0.045960 -0.070750      12        1   0.000460
16    -0.045960 -0.070750      13        1   0.000449
6     -0.045960 -0.070750      14        1   0.000449
22    -0.045960 -0.070750      15        1   0.000447
0     -0.045960 -0.070750      16        1   0.000442
11    -0.045960 -0.070750      17        1   0.000440
14    -0.045960 -0.070750      18        1   0.000429
21    -0.045960 -0.070750      19        1   0.000428
1     -0.045960 -0.070750      20        1   0.000422
5     -0.045960 -0.070750      21        1   0.000420
29    -0.045960 -0.070750      22        1   0.000419
10    -0.045960 -0.070750      23        1   0.000418
24    -0.045960 -0.070750      24        1   0.000417
15    -0.045960 -0.070750      25        1   0.000417
27    -0.045960 -0.070750      26        1   0.000416
20    -0.045960 -0.070750      27        1   0.000415
4     -0.045960 -0.070750      28        1   0.000414
25    -0.045960 -0.070750      29        1   0.000414
3     -0.045960 -0.070750      30        1   0.000413, topic_info=        Term          Freq         Total Category  logprob  loglift
695     shit   8276.000000   8276.000000  Default  30.0000  30.0000
669      say  15388.000000  15388.000000  Default  29.0000  29.0000
582   people  13621.000000  13621.000000  Default  28.0000  28.0000
483      man   8655.000000   8655.000000  Default  27.0000  27.0000
321  fucking   4297.000000   4297.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
428      kid      0.000125   4591.840407  Topic30 -10.4276  -5.0195
432     know      0.000125   3202.729520  Topic30 -10.4276  -4.6592
451     life      0.000125   4260.878815  Topic30 -10.4276  -4.9447
459   little      0.000125   6100.374246  Topic30 -10.4276  -5.3036
466     look      0.000125   4805.501705  Topic30 -10.4276  -5.0650

[1927 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
4927      1  0.252439  abortion
4927      3  0.742468  abortion
8         1  0.592132  actually
8         2  0.342785  actually
8         3  0.057665  actually
...     ...       ...       ...
864       4  0.023966      year
864       5  0.022169      year
1363      1  0.998138      yell
1363      3  0.002452      yell
1365      4  0.976440     zebra

[968 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 27, 14, 10, 8, 18, 13, 29, 24, 3, 9, 19, 17, 7, 23, 1, 12, 15, 22, 2, 6, 30, 11, 25, 16, 28, 21, 5, 26, 4])

/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__)

/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__)